In [1]:
import h5py, os
from functions import transforms as T
from functions.subsample import MaskFunc

import numpy as np
from scipy.io import loadmat
from matplotlib import pyplot as plt

import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import DataLoader

In [2]:
def show_slices(data, slice_nums, cmap=None): # visualisation
    fig = plt.figure(figsize=(15,10)) # default figure size
    for i, num in enumerate(slice_nums): # iterate over ordinated slices
        plt.subplot(1, len(slice_nums), i + 1) # set up i+1-th subplot in a grid with height 1 and width len
        plt.imshow(data[num], cmap=cmap) # plot the data of each slice as cmap
        plt.axis('off')

In [3]:
def load_data_path(train_data_path, val_data_path):
    """ Go through each subset (training, validation) and list all 
    the file names, the file paths and the slices of subjects in the training and validation sets 
    """

    data_list = {} # initiate the data list
    train_and_val = ['train', 'val'] # set the keys in data list
    data_path = [train_data_path, val_data_path] # import the data path for both training and validation
      
    for i in range(len(data_path)): # iterate over both paths

        data_list[train_and_val[i]] = [] # reset the data list 
        
        which_data_path = data_path[i] # set the data path to current dataset
    
        for fname in sorted(os.listdir(which_data_path)): # iterate over all listed files in the data path
            
            subject_data_path = os.path.join(which_data_path, fname) # set subject data path with its directory
                     
            if not os.path.isfile(subject_data_path): continue # iterate until all files are registered on path
            
            with h5py.File(subject_data_path, 'r') as data: # open the subject at the desinated path
                num_slice = data['kspace'].shape[0] # set # of slices as the 1st-D of the data's kspace data
                
            # the first 5 slices are mostly noise so it is better to exlude them
            data_list[train_and_val[i]] += [(fname, subject_data_path, slice) for slice in range(5, num_slice)]
            # data_list is concatenated by a list each consisted with a file's name, path and slices excluding 
            # the first 5 slices that are  mostly considered as noise 
            
    return data_list # {'train': [('file1000000.h5','/home/kevinxu/Documents/NC2019MRI/train/file1000000.h5',5)

In [4]:
class MRIDataset(DataLoader):
    def __init__(self, data_list, acceleration, center_fraction, use_seed):
        self.data_list = data_list # data list imported to be processed
        self.acceleration = acceleration # defined AF
        self.center_fraction = center_fraction # defined CF
        self.use_seed = use_seed # defined random or stable mask

    def __len__(self): # create an attribute of the length of dataset
        return len(self.data_list) 

    def __getitem__(self, idx): # return an item when called with an index
        subject_id = self.data_list[idx] # set subject id as the called index
        
        # return ground truth, under-sampled real image, masked kspace data, masks and norm
        return get_epoch_batch(subject_id, self.acceleration, self.center_fraction, self.use_seed)

In [5]:
def get_epoch_batch(subject_id, acc, center_fract, use_seed=True):
    ''' random select a few slices (batch_size) from each volume'''

    fname, rawdata_path, slice = subject_id # read file name, path, slice
    
    with h5py.File(rawdata_path, 'r') as data:
        rawdata = data['kspace'][slice] # return a random slice of the kspace data
                      
    slice_kspace = T.to_tensor(rawdata).unsqueeze(0) # transform the slice to tensor and add a dimension
    S, Ny, Nx, ps = slice_kspace.shape # assign parameters according to the kspace shape; S = batch_size

    # apply random mask
    shape = np.array(slice_kspace.shape) 
    mask_func = MaskFunc(center_fractions=[center_fract], accelerations=[acc]) # see subsample.py
    seed = None if not use_seed else tuple(map(ord, fname)) # generate a tuple of fname unicode for seeding
    mask = mask_func(shape, seed) # generate the mask as the shape [1,1,num_col,1]
      
    # undersample
    masked_kspace = torch.where(mask == 0, torch.Tensor([0]), slice_kspace) # apply mask to kspace slice
    masks = mask.repeat(S, Ny, 1, ps) # save the mask and expand its shape to fit sample [S, Ny, num_col, ps]

    img_gt, img_und = T.ifft2(slice_kspace), T.ifft2(masked_kspace) # ground truth and under-sampled image

    ## perform data normalization which is important for network to learn useful features
    ## during inference there is no ground truth image so use the zero-filled recon to normalize
    norm = T.complex_abs(img_und).max() # set norm as the max value of sqrt((img_und)**2)
    if norm < 1e-6: norm = 1e-6 # avoid value error of being divided by zero
    
    # normalized data: ground truth, under-sampled real image, masked kspace data, respectively
    img_gt, img_und, masked_kspace = img_gt/norm, img_und/norm, masked_kspace/norm 
    
    # return ground truth, under-sampled real image, masked kspace data, masks and norm
    return img_gt.squeeze(0), img_und.squeeze(0), masked_kspace.squeeze(0), masks.squeeze(0), norm

In [6]:
if __name__ == '__main__': # main() function of the file
    
    data_path_train = '/home/kevinxu/Documents/NC2019MRI/train'
    data_path_val = '/home/kevinxu/Documents/NC2019MRI/train'
    data_list = load_data_path(data_path_train, data_path_val) # first load all file names, paths and slices.

    acc = 8 # AF
    cen_fract = 0.04 # central fraction that are set to be included
    seed = False # random masks for each slice 
    num_batch = 1 # batch size of each loading process
    num_workers = 12 # data loading is faster using a bigger number for num_workers. 0 means using cpu.
    
    # create data loader for training set. It applies same to validation set as well
    train_dataset = MRIDataset(data_list['train'], acceleration=acc, center_fraction=cen_fract, use_seed=seed)
    train_loader = DataLoader(train_dataset, shuffle=True, batch_size=num_batch, num_workers=num_workers) 
    for iteration, sample in enumerate(train_loader): # iterate over every element in the train_loader
        img_gt, img_und, masked_kspace, masks, norm = sample
        
        # stack different slices into a volume for visualisation
        A = masks[...,0].squeeze()    # mask applied
        B = torch.log(T.complex_abs(masked_kspace) + 1e-9).squeeze()    # masked kspace image
        C = T.center_crop(T.complex_abs(img_und), [320,320])    # image generated from under-sampled data
        D = T.complex_abs(img_gt).squeeze()    # image generated from ground truth data
        # all_imgs = torch.stack([A,B,C,D], dim=0)


In [7]:
print(A.shape)
print(B.shape)
print(C.shape)
print(D.shape)

torch.Size([640, 368])
torch.Size([640, 368])
torch.Size([1, 320, 320])
torch.Size([640, 368])


In [8]:
class ConvBlock(nn.Module): # the cnn block used in u-net architecture
    """
    A Convolutional Block that consists of two convolution layers each followed by
    instance normalization, relu activation and dropout.
    """

    def __init__(self, in_chans, out_chans, drop_prob):
        """
        Args:
            in_chans (int): Number of channels in the input.
            out_chans (int): Number of channels in the output.
            drop_prob (float): Dropout probability.
        """
        super().__init__()

        self.in_chans = in_chans
        self.out_chans = out_chans
        self.drop_prob = drop_prob # parameter defining the probability of dropout in each layer

        self.layers = nn.Sequential( # create layers
            nn.Conv2d(in_chans, out_chans, kernel_size=3, padding=1), # convolution layer
            nn.InstanceNorm2d(out_chans), # instance normalization
            nn.ReLU(), # relu activation
            nn.Dropout2d(drop_prob), # dropout regularisation
            nn.Conv2d(out_chans, out_chans, kernel_size=3, padding=1), # the second layer
            nn.InstanceNorm2d(out_chans),
            nn.ReLU(),
            nn.Dropout2d(drop_prob)
        )

    def forward(self, input):
        """
        Args:
            input (torch.Tensor): Input tensor of shape [batch_size, self.in_chans, height, width]

        Returns:
            (torch.Tensor): Output tensor of shape [batch_size, self.out_chans, height, width]
        """
        return self.layers(input)

    def __repr__(self):
        return f'ConvBlock(in_chans={self.in_chans}, out_chans={self.out_chans}, ' \
            f'drop_prob={self.drop_prob})'


class UnetModel(nn.Module):
    """
    PyTorch implementation of a U-Net model.

    This is based on:
        Olaf Ronneberger, Philipp Fischer, and Thomas Brox. U-net: Convolutional networks
        for biomedical image segmentation. In International Conference on Medical image
        computing and computer-assisted intervention, pages 234–241. Springer, 2015.
    """

    def __init__(self, in_chans, out_chans, chans, num_pool_layers, drop_prob):
        """
        Args:
            in_chans (int): Number of channels in the input to the U-Net model.
            out_chans (int): Number of channels in the output to the U-Net model.
            chans (int): Number of output channels of the first convolution layer.
            num_pool_layers (int): Number of down-sampling and up-sampling layers.
            drop_prob (float): Dropout probability.
        """
        super().__init__()

        self.in_chans = in_chans
        self.out_chans = out_chans
        self.chans = chans
        self.num_pool_layers = num_pool_layers
        self.drop_prob = drop_prob

        # down-sample cnn layers
        self.down_sample_layers = nn.ModuleList([ConvBlock(in_chans, chans, drop_prob)])
        ch = chans # number of output channels of the first convolution layer
        for i in range(num_pool_layers - 1): # iterate over number of total pooling layers
            self.down_sample_layers += [ConvBlock(ch, ch * 2, drop_prob)] # update down-sample layers x2
            ch *= 2 # double the output channels of the cnn layers
        self.conv = ConvBlock(ch, ch, drop_prob) # update conv layer with ch

        # up-sample cnn layers
        self.up_sample_layers = nn.ModuleList()
        for i in range(num_pool_layers - 1):
            self.up_sample_layers += [ConvBlock(ch * 2, ch // 2, drop_prob)] # update up-sample layers //4
            ch //= 2 # update ch with ch floordiv by 2
        self.up_sample_layers += [ConvBlock(ch * 2, ch, drop_prob)] # update up-sample layers //2
        self.conv2 = nn.Sequential(
            nn.Conv2d(ch, ch // 2, kernel_size=1),
            nn.Conv2d(ch // 2, out_chans, kernel_size=1), 
            nn.Conv2d(out_chans, out_chans, kernel_size=1), # conv filter with output shape
        )

    def forward(self, input):
        """
        Args:
            input (torch.Tensor): Input tensor of shape [batch_size, self.in_chans, height, width]

        Returns:
            (torch.Tensor): Output tensor of shape [batch_size, self.out_chans, height, width]
        """
        stack = []
        output = input
        # Apply down-sampling layers
        for layer in self.down_sample_layers:
            output = layer(output) # fit layers
            stack.append(output) # append new output in every iteration to stack []
            output = F.max_pool2d(output, kernel_size=2) # apply max-pool

        output = self.conv(output) # apply conv block

        # Apply up-sampling layers
        for layer in self.up_sample_layers:
            output = F.interpolate(output, scale_factor=2, mode='bilinear', align_corners=False) # upsample
            output = torch.cat([output, stack.pop()], dim=1) # concatenate output with the last stack on dim=1
            output = layer(output) # fit layers
        return self.conv2(output) # apply up-sampling layers


In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UnetModel(in_chans=1, out_chans=1, chans=32, num_pool_layers=4, drop_prob=0.5).to(device)
optimizer = optim.SGD(params=model.parameters(), lr=0.01)

def train(epoch):
    model.train()
    loss_count = []
    for iteration, sample in enumerate(train_loader):
        img_gt, img_und, masked_kspace, masks, norm = sample
        
        img_und = T.complex_abs(img_und)
        img_gt = T.complex_abs(img_gt)
        
        img_und = img_und.unsqueeze(0)
        img_gt = img_gt.unsqueeze(0)
       
        img_und = T.center_crop(img_und, [320, 320])
        img_gt = T.center_crop(img_gt, [320, 320])
        
        X = img_und
        Y = img_gt
        X, Y = X.to(device), Y.to(device)
        
        output = model(X)
        loss = F.l1_loss(output, Y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_count.append(loss)
        print(loss.item())

In [ ]:
for epoch in range(1, 5 + 1):
    train(epoch)

0.1451922059059143
0.13399459421634674
0.226047083735466
0.18528619408607483
0.22873082756996155
0.162269726395607
0.1423427164554596
0.1563226729631424
0.10322114825248718
0.14555524289608002
0.1403808295726776
0.16193753480911255
0.2081247717142105
0.13759863376617432
0.25448372960090637
0.23718616366386414
0.10618693381547928
0.09569073468446732
0.17723172903060913
0.13784416019916534
0.18521665036678314
0.22095516324043274
0.19658885896205902
0.14369629323482513
0.23650585114955902
0.14083686470985413
0.15121260285377502
0.18356284499168396
0.20753280818462372
0.22143849730491638
0.21853795647621155
0.15041939914226532
0.20778389275074005
0.1366192102432251
0.17474672198295593
0.14438188076019287
0.1215292364358902
0.13965822756290436
0.2827502191066742
0.24032673239707947
0.16718004643917084
0.1054491251707077
0.1708657145500183
0.15838216245174408
0.10379153490066528
0.1550397276878357
0.1401236653327942
0.2539450526237488
0.20414745807647705
0.22857467830181122
0.233491539955139

0.1524774581193924
0.1370285004377365
0.08900697529315948
0.142831951379776
0.12230705469846725
0.22308143973350525
0.19997093081474304
0.23094642162322998
0.14084650576114655
0.12820014357566833
0.13570408523082733
0.1327357143163681
0.10806754231452942
0.10209755599498749
0.08114292472600937
0.12111146748065948
0.09899626672267914
0.09874363988637924
0.10925010591745377
0.0940852165222168
0.1796237975358963
0.1672094762325287
0.17968136072158813
0.09341632574796677
0.08765564113855362
0.15079770982265472
0.12796525657176971
0.08115202188491821
0.1462436318397522
0.15172742307186127
0.1570253223180771
0.12998832762241364
0.11610017716884613
0.12353706359863281
0.1077425479888916
0.10800345242023468
0.103665791451931
0.18247123062610626
0.1459828019142151
0.11274610459804535
0.13646499812602997
0.09665622562170029
0.12567879259586334
0.1730988472700119
0.1403058022260666
0.11699961870908737
0.163601815700531
0.25615614652633667
0.12799270451068878
0.13245010375976562
0.1177488043904304